<b>Getting the imports</b>

In [22]:
import os
import shutil
import random
import math
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator


<b>Downlad the RGB Dataset from:</b><br></br>
<a>https://github.com/ardamavi/Sign-Language-Digits-Dataset</a>

<b>Oganizing Images from the Dataset as Required by the NN</b><br></br>
<h>Setting up "train", "test" and "valid" Directories and populating them with the data from the dataset</h>

In [ ]:
#Function for creating the directories of different classes 
#Takes train, test or valid as input
def create_dir_for_signlanguage(dir_name):
    if os.path.isdir(dir_name) is False: 
        print("creating", dir_name)
        os.mkdir(# YOUR ABSOLUTE DIR PATHN WHERE YOU WANT TO CREATE THIS DIRECTORY # + '\\' + dir_name)
        os.chdir(dir_name)
        for i in range(10):
            os.mkdir(f'{i}')
        print(dir_name, "created!")
        os.chdir('..')
    else:
        print(dir_name, "already exists!")

In [127]:
#Function to get list of files in a directory
def get_list_files(dir_path):
    list_files = next(os.walk(dir_path))[2] #dir_path is your directory path as string
    return list_files

In [129]:
#Function to copy images from datasets to relevant directories acc. to given test, train and valid ratios
def dataset_to_dir(dataset_dir, dir_class, target_dir, train_ratio, test_ratio, valid_ratio):
    if (train_ratio + test_ratio + valid_ratio > 1):
        print("Invalid Ratios!")
    else:
        dataset_dir = dataset_dir + '\\' + dir_class
        list_files = get_list_files(dataset_dir)
        total_num_files = len(list_files)
        counter = 0
                
        num_files_to_transfer_to_train = math.floor(total_num_files * train_ratio)
        num_files_to_transfer_to_test = math.floor(total_num_files * test_ratio)
        num_files_to_transfer_to_valid = math.floor(total_num_files * valid_ratio)
        
        for i in range(total_num_files):
            #Transfering file to Train Acc. to given Ratio
            if  (counter > 0 and counter < num_files_to_transfer_to_train):                
                src = dataset_dir + '\\' + list_files[counter] 
                dest = target_dir + f'\\train\\{dir_class}\\' + list_files[counter]
                shutil.copyfile(src, dest)
                counter += 1
            
            #Transfering file to Test Acc. to given Ratio
            elif (counter > num_files_to_transfer_to_train and \
                  counter <= num_files_to_transfer_to_train + num_files_to_transfer_to_test):                
                src = dataset_dir + '\\' + list_files[counter] 
                dest = target_dir + f'\\test\\{dir_class}\\' + list_files[counter]
                shutil.copyfile(src, dest)
                counter += 1
                
            #Transfering file to Valid Acc. to given Ratio                
            else:                
                src = dataset_dir + '\\' + list_files[counter] 
                dest = target_dir + f'\\valid\\{dir_class}\\' + list_files[counter]
                shutil.copyfile(src, dest)
                counter += 1

<b>Creating the Directories</b>

In [122]:
os.chdir("# YOUR ABSOLUTE DIR PATHN WHERE YOU'RE WORKING #")
create_dir_for_signlanguage("train")
create_dir_for_signlanguage("test")
create_dir_for_signlanguage("valid")

train already exists!
test already exists!
valid already exists!


<b>Copying Data into the Directories</b>

In [130]:
os.chdir("# YOUR ABSOLUTE DIR PATHN WHERE YOU'RE WORKING #")

#Calling the function for all 10 classes
for i in range(10): 
    dataset_to_dir("# YOUR ABSOLUTE DIR PATHN WHERE YOU'RE WORKING #" + "\\" + "Dataset",
                   f"{i}",
                   "# YOUR ABSOLUTE DIR PATHN WHERE YOU'RE WORKING #",
                  0.7, 0.15, 0.15)

<b>Downloading the MobileNet Model</b>

In [141]:
mobilenet_model = MobileNet()

17227776/17225924 [==============================] - 100s 6us/step


<b>Saving the Model</b>

In [142]:
mobilenet_model.save("MobileNet.h5")

<b>Loading the Model</b>

In [2]:
mobilenet_model = load_model("MobileNet.h5")

<b>Fine Tunning the Model</b>

In [10]:
x = mobilenet_model.layers[-6].output
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=mobilenet_model.input, outputs=predictions)

In [12]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

<b>Saving the Fine Tunned Model</b>

In [14]:
model.save("Fine Tunned MobileNet.h5")

<b>Loading and Compiling the Fine Tunned Model</b>

In [18]:
model = load_model("FIne Tunned MobileNet.h5")
print("Compiling Model...")
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
print("Model Compiled Successfuly!")

Compiling Model...
Model Compiled Successfuly!


<b>Training the Model</b>

In [19]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [23]:
generator = ImageDataGenerator(
                            rotation_range=15, 
                            horizontal_flip=True,
                            #vertical_flip=True, 
                            width_shift_range=0.1, 
                            height_shift_range=0.1, 
                            zoom_range=0.2,
                            #channel_shift_range=50
                            )

In [24]:
train_batches = generator.flow_from_directory(train_path, target_size=(224, 244), classes=classes, batch_size=16)
valid_batches = generator.flow_from_directory(valid_path, target_size=(224, 244), classes=classes, batch_size=8)
test_batches = generator.flow_from_directory(test_path, target_size=(224, 244), classes=classes, batch_size=8)

Found 1428 images belonging to 10 classes.
Found 330 images belonging to 10 classes.
Found 304 images belonging to 10 classes.


In [ ]:
model.fit(train_batches, steps_per_epoch=18, epochs=60,
                   validation_data=valid_batches, validation_steps=3, verbose=2)

<b>Saving the Model to disk</b>

In [ ]:
model.save("finetunned_trained.h5")